In [1]:
import collections
import string
import itertools
import re

In [2]:
Link = collections.namedtuple('Link', 'height left right')

In [3]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [4]:
def read_net_string(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [5]:
def read_net(filename):
    with open(filename) as f:
        pairs = [re.split('\D+', p.strip()) for p in f.readlines()]
        lrs = [(int(lr[1]), int(lr[2])) for lr in pairs]
        return [Link(h, l, r) 
                for h, (l, r) in enumerate(lrs)]

In [6]:
small_net = read_net('04-small.txt')
small_net

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [7]:
net = read_net('04-lines.txt')
len(net)

10135

In [8]:
def show_net(links, pair_sep=', '):
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [9]:
def link_ends(link):
    return set((link.left, link.right))

In [10]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [11]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [12]:
max(l.height for l in small_net)

14

In [13]:
max(l.height for l in pack(small_net))

10

In [14]:
max(l.height for l in net)

10134

In [15]:
max(l.height for l in pack(net))

2286

In [16]:
def height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [17]:
def follow_many(in_sequence, net):
    hgs = height_groups(net)
    seq = list(in_sequence)
    for h in hgs:
        for link in hgs[h]:
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [18]:
''.join(follow_many('abcdefghij', small_net))

'acfbedghij'

In [19]:
%%timeit
follow_many('abcdefghij', small_net)

10000 loops, best of 3: 38.1 µs per loop


In [20]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [21]:
%%timeit
follow_many(string.ascii_lowercase, net)

100 loops, best of 3: 19 ms per loop


In [22]:
pnet = pack(net)

In [23]:
def eliminable_pairs(net):
    hgs = height_groups(net)
    eps = []
    for h in range(1, max(hgs.keys())):
        for l in hgs[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in hgs[h-1]:
                eps += [(l, o)]
    return eps

In [24]:
eliminable_pairs(pnet)

[(Link(height=115, left=2, right=23), Link(height=114, left=2, right=23)),
 (Link(height=149, left=15, right=23), Link(height=148, left=15, right=23)),
 (Link(height=201, left=0, right=7), Link(height=200, left=0, right=7)),
 (Link(height=207, left=22, right=23), Link(height=206, left=22, right=23)),
 (Link(height=210, left=17, right=23), Link(height=209, left=17, right=23)),
 (Link(height=247, left=16, right=20), Link(height=246, left=16, right=20)),
 (Link(height=418, left=19, right=24), Link(height=417, left=19, right=24)),
 (Link(height=424, left=9, right=21), Link(height=423, left=9, right=21)),
 (Link(height=453, left=3, right=16), Link(height=452, left=3, right=16)),
 (Link(height=456, left=2, right=22), Link(height=455, left=2, right=22)),
 (Link(height=465, left=18, right=20), Link(height=464, left=18, right=20)),
 (Link(height=491, left=14, right=18), Link(height=490, left=14, right=18)),
 (Link(height=552, left=16, right=17), Link(height=551, left=16, right=17)),
 (Link(heig

In [25]:
%%timeit
eliminable_pairs(pnet)

10 loops, best of 3: 23.5 ms per loop


In [26]:
def eliminable_pair(hgs):
    for h in range(1, max(hgs.keys())):
        for l in hgs[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in hgs[h-1]:
                return l, o
    return None

In [27]:
def eliminate_pairs(net):
    hgs = height_groups(pack(net))
    eliminable_links = eliminable_pair(hgs)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        hgs = height_groups(pack(net))
        eliminable_links = eliminable_pair(hgs)
    return net

In [28]:
enet = eliminate_pairs(pnet)

In [29]:
assert follow_many(string.ascii_lowercase,  net) == follow_many(string.ascii_lowercase, enet)
assert follow_many(string.ascii_lowercase, pnet) == follow_many(string.ascii_lowercase, enet)

In [30]:
%%timeit
eliminate_pairs(pnet)

1 loop, best of 3: 2.35 s per loop


In [31]:
def triple_pair(height_groups, debug=False):
    ts = []
    for h in range(3, max(height_groups.keys())):
        for d in height_groups[h]:
            if debug: print('d:', d)
            ch = h - 1
            cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
            if debug: print('cs:', cs)
            while ch > 2 and not cs:
                ch -= 1
                cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
                if debug: print('cs:', cs)
            if len(cs) == 1:
                c = cs[0]
                lines = set((d.left, d.right, c.left, c.right))
                if debug: print('c:', '; lines:', lines)
                bs = [l for l in height_groups[ch-1] if link_ends(l) & lines]
                b = Link(ch - 1, d.left, d.right)
                if debug: print('b:', b, '; bs:', bs)
                if len(bs) == 1 and b in bs:
                    ah = b.height - 1
                    als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                    if debug: print('ah:', ah, '; als:', als)
                    while ah > 0 and not als:
                        ah -= 1
                        als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                        if debug: print('ah:', ah, '; als:', als)
                    a = Link(ah, c.left, c.right)
                    if debug: print('a:', a)
                    if a in als:
                        if debug: print('adding:', a, b, c, d)
                        ts += [(a, b, c, d)]
    return ts

In [32]:
def eliminate_a_triple_pair(net, debug=False):
    hgs = height_groups(net)

    tps = triple_pair(hgs)
    if debug: print('eatp', tps)
    if tps:
        a, b, c, d = tps[0]
        x = Link(b.height - 0.5, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        if debug: print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [33]:
hgs = height_groups(enet)
triple_pair(hgs)

[(Link(height=8, left=1, right=5),
  Link(height=9, left=1, right=21),
  Link(height=10, left=1, right=5),
  Link(height=11, left=1, right=21)),
 (Link(height=40, left=16, right=23),
  Link(height=41, left=16, right=19),
  Link(height=42, left=16, right=23),
  Link(height=43, left=16, right=19)),
 (Link(height=62, left=0, right=10),
  Link(height=63, left=10, right=13),
  Link(height=64, left=0, right=10),
  Link(height=65, left=10, right=13)),
 (Link(height=137, left=23, right=24),
  Link(height=139, left=0, right=24),
  Link(height=140, left=23, right=24),
  Link(height=141, left=0, right=24)),
 (Link(height=138, left=10, right=21),
  Link(height=139, left=2, right=10),
  Link(height=140, left=10, right=21),
  Link(height=141, left=2, right=10)),
 (Link(height=139, left=2, right=10),
  Link(height=140, left=10, right=21),
  Link(height=141, left=2, right=10),
  Link(height=142, left=10, right=21)),
 (Link(height=156, left=6, right=11),
  Link(height=157, left=3, right=6),
  Link(heig

In [34]:
etnet = eliminate_a_triple_pair(enet)

In [35]:
assert follow_many(string.ascii_lowercase, etnet) == follow_many(string.ascii_lowercase, enet)

In [36]:
def eliminate_triple_pairs(net):
    print(len(net))
    new_net = eliminate_a_triple_pair(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair(net)
    return net

In [37]:
setnet = eliminate_triple_pairs(enet)

10033
10033
10031
10029
10027
10025
10023
10021
10019
10017
10015
10013
10011
10009
10007
10005
10003
10001
9999
9997
9995
9993
9991
9989
9987
9985
9983
9981
9979
9977
9975
9973
9971
9969
9967
9965
9963
9961
9959
9957
9955
9953
9951
9949
9947
9945
9943
9941
9939


In [38]:
len(setnet)

9937

In [39]:
assert follow_many(string.ascii_lowercase, etnet) == follow_many(string.ascii_lowercase, enet)

In [40]:
''.join(follow_many(string.ascii_lowercase, etnet))

'doqzmbishkwunvltpcexyjgfra'

In [41]:
''.join(follow_many(string.ascii_lowercase, setnet))

'doqzmbishkwunvltpcexyjgfra'

In [42]:
''.join(follow_many(string.ascii_lowercase, enet))

'doqzmbishkwunvltpcexyjgfra'

In [43]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [44]:
eliminable_pairs(etnet)

[]

In [45]:
len(net), len(etnet)

(10135, 10031)

In [46]:
def simplify(net0):
    netp = eliminate_pairs(net0)
    new_net = eliminate_a_triple_pair(netp)
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        new_net = eliminate_a_triple_pair(netp)
    return netp

In [47]:
simple_net = simplify(pnet)

In [48]:
''.join(follow_many(string.ascii_lowercase, simple_net)) == ''.join(follow_many(string.ascii_lowercase, net))

True

In [49]:
''.join(follow_many(string.ascii_lowercase, simple_net))

'doqzmbishkwunvltpcexyjgfra'

In [50]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [51]:
len(simple_net)

9931

In [52]:
def simplify_with_checks(net0):
    netp = eliminate_pairs(net0)
    if follow_many(string.ascii_lowercase, net0) != follow_many(string.ascii_lowercase, netp):
        print('pairs', eliminable_pairs(net0))
        return net0
    else:
        print('pairs ok')
    new_net = eliminate_a_triple_pair(netp)
    if new_net and follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
        hg = find_height_groups(netp)
        print('triple', triple_pair_hg(hg))
        return netp
    else:
        print('triple ok')
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        if follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
            print('pairs', eliminable_pairs(new_net))
            return new_net
        else:
            print('pairs ok')
        new_net = eliminate_a_triple_pair(netp)
        if new_net and follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
            hg = find_height_groups(netp)
            print('triple', triple_pair_hg(hg))
            return netp
        else:
            print('triple ok')
    print('** done')
    return netp

In [53]:
spnet = simplify_with_checks(pnet)

pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
** done


In [54]:
''.join(follow_many(string.ascii_lowercase, spnet)) == ''.join(follow_many(string.ascii_lowercase, net))

True

In [55]:
len(spnet)

9931